In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
# Note: this notebook has not been re-run with this patch version yet.
# That means that the current plots are still based on version-001-p00
selection_version = 'version-001-p01' 

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/home/mhuennefeld/public_html/analyses/DNNCascade/plots/review/gp_model_confusion'
    
elif 'phobos' in host_name:
    print('Working on Phobos')
    data_prefix = '/net/big-tank/POOL/users/mhuennefeld/analyses/DNNCascade/data/cscd/final'
    ana_dir = '/net/big-tank/POOL/users/mhuennefeld/analyses/DNNCascade/csky/analyses/'
    plot_dir = '/home/mhuennefeld/analyses/DNNCascade/plots/review/gp_model_confusion'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Create new templates

#### Disc Template

In [ ]:
import healpy as hp

nside = 128
width_deg = 1
smearing_deg = 5
template_disc_gal = np.zeros(hp.nside2npix(nside))
pix_disc = hp.query_strip(nside=nside, theta1=np.pi/2.-np.deg2rad(width_deg), theta2=np.pi/2. + np.deg2rad(width_deg))
template_disc_gal[pix_disc] = 1.

# smear with Gaussian
template_disc_gal = hp.smoothing(template_disc_gal, sigma=np.deg2rad(smearing_deg))

# transform from galactic to equatorial coordinates
r = hp.Rotator(coord='GC')  # Transforms galactic to equatorial coordinates
theta_gal, phi_gal = hp.pix2ang(nside, np.arange(hp.nside2npix(nside))) #theta, phi
phi_gal = 1.754*np.pi - phi_gal # no clue why this makes it align....
theta_ecl, phi_ecl = r(theta_gal, phi_gal)  # Apply the conversion
print('phi_ecl', phi_ecl)
print(np.max(phi_ecl), np.min(phi_ecl))

template_disc = healpy.get_interp_val(template_disc_gal, theta_ecl, phi_ecl)

#skyplot = cy.plotting.SkyPlot(m=np.log10(template_disc), title='Template: GalacticPlane', unit=r'$\log_{10}(PDF)$')
skyplot = cy.plotting.SkyPlot(m=template_disc, title='Template: GalacticPlane', unit=r'PDF')
skyplot.show_gp()
skyplot.show_gc()
skyplot.fig.tight_layout()
skyplot.fig.savefig('{}/skymap_template_gp.png'.format(plot_dir))

#### GC template

In [ ]:
import healpy as hp

nside = 128
smearing_deg = 5

r = hp.Rotator(coord='GC')  # Transforms galactic to equatorial coordinates
theta, phi = r(np.pi/2., 0.)  # Apply the conversion
template_gc = np.zeros(hp.nside2npix(nside))
template_gc[hp.ang2pix(nside=nside, theta=theta, phi=phi)] = 1.

# smear with Gaussian
template_gc = hp.smoothing(template_gc, sigma=np.deg2rad(smearing_deg))
floor = 1e-7
template_gc[template_gc < floor] = floor

#skyplot = cy.plotting.SkyPlot(m=np.log10(template_gc), title='Template: GalacticCenter', unit=r'$\log_{10}(PDF)$')
skyplot = cy.plotting.SkyPlot(m=template_gc, title='Template: GalacticCenter', unit=r'PDF')
skyplot.show_gp()
skyplot.show_gc()
skyplot.fig.tight_layout()
skyplot.fig.savefig('{}/skymap_template_gc.png'.format(plot_dir))


In [ ]:
template_pi0 = cg.template_repo.get_template('Fermi-LAT_pi0_map')

skyplot = cy.plotting.SkyPlot(m=np.log10(template_pi0))
#skyplot = cy.plotting.SkyPlot(m=template_pi0)
skyplot.show_gp()
skyplot.show_gc()

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/baseline_analysis__v-1.0.0-rc1/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get trial runners

In [ ]:
tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
    'GalacticCenter': get_template_tr(template_gc),
    'GalacticPlane': get_template_tr(template_disc),
    'pi0_gamma2': get_template_tr(template_pi0, gamma=2.),
    'pi0_gamma3_10TeV': get_template_tr(template_pi0, gamma=3., cutoff_tev=10),
}

#### Get bkg fits for each template

In [ ]:
bkg_file_dict = {
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if key in bkg_file_dict:
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        bkg_dict[key] = cy.dists.TSD(sig['poisson']['nsig'][0.0]['ts'])
    
    else:
        print('Running background trials for template {}'.format(key))
        bkg_dict[key] = cy.dists.Chi2TSD(tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20))
        

In [ ]:
for key, bg in bkg_dict.items():
    fig, ax = plot_bkg_trials(bg)
    ax.set_title('Template: {}'.format(key))
    ax.set_yscale('log')
    

#### Define n-sigma ns for each template

In [ ]:
nsigma = 5

if nsigma == 5:
    ns_dict = {
        'pi0': 752, #540,
        'kra5': 281, #230,
        'kra50': 233, #278,
    }
    
elif nsigma == 4:
    ns_dict = {
        'pi0': 590,
        'kra5': 222,
        'kra50': 184,
    }
    
elif nsigma == 3:
    ns_dict = {
        'pi0': 433,
        'kra5': 161,
        'kra50': 133,
    }
else:
    raise ValueError(nsigma)

print('Using nsigma = ', nsigma)

#### Compute nsigma for new templates

In [ ]:
for key, tr in tr_dict.items():
    if key not in ns_dict:
        print('Computing ns for {} sigma for template {}'.format(nsigma, key))
        
        template_sens = tr.find_n_sig(
            bkg_dict[key].isf_nsigma(nsigma),
            0.5, #percent above threshold (0.5 for dp)
            n_sig_step=50,
            batch_size = 1000, 
            tol = 0.02,
        )
        ns_dict[key] = template_sens['n_sig']
        

In [ ]:
ns_dict

#### Get trials for each template

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm

n_trials = 100
seed = 42
trials_dict = {}

for key, tr in tr_dict.items():
    
    n_sig = ns_dict[key]
    trials = []
    
    print('Injecting {} signal events for template {}'.format(n_sig, key))
    for i in tqdm(range(n_trials), total=n_trials):
        trials.append(tr.get_one_trial(n_sig=n_sig, poisson=True, seed=seed + i))
    
    trials_dict[key] = trials
    

#### Get fits for each template combination

In [ ]:
ts_dict = {}
for key_inj, tr_inj in tr_dict.items():
    
    for key, tr in tr_dict.items():
        print('Computing TS values for injection {} and testing with {}'.format(key_inj, key))

        ts_values = []
        n_values = len(trials_dict[key_inj])
        for trial in tqdm(trials_dict[key_inj], total=n_values):
            ts_values.append(tr.get_one_fit_from_trial(trial)) 
        
        ts_values = np.array(ts_values)
        ts_dict[(key_inj, key)] = cy.utils.Arrays({
          'ts': ts_values[:, 0],  
          'ns': ts_values[:, 1],  
        })




#### Compute significance

In [ ]:
p_val_dict = {}
sigma_dict = {}
for key, ts_values in ts_dict.items():
    bg = bkg_dict[key[1]]
    max_bg_ts = np.max(bg.values)
    mask_above = ts_values.ts > max_bg_ts
    ts = np.array(ts_values.ts)
    if np.sum(mask_above) > 0:
        print('Setting {} ts values to max bkg ts value of {}.'.format(
            np.sum(mask_above), max_bg_ts))
        ts[mask_above] = max_bg_ts
    p_val_dict[key] = bg.sf(ts)
    sigma_dict[key] = bg.sf_nsigma(ts)


#### Plot Distributions

In [ ]:
from itertools import cycle

def plot_model_confusion(template_keys, testing_keys=None, bins=np.linspace(0, 6, 20)):
    n_keys = len(template_keys)
    
    if testing_keys is None:
        testing_keys = template_keys
        
    fig, axes = plt.subplots(n_keys, 1, figsize=(9, 3*n_keys), sharex=True)
    
    if n_keys <= 5:
        color_cycle = cycle(soft_colors)
    else:
        color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

    for i, key_inj in enumerate(template_keys):

        ax = axes[i]
        ax.set_title('Injection template: {}'.format(key_inj))

        for j, key in enumerate(testing_keys):
            sigmas = sigma_dict[(key_inj, key)]
            if not np.isfinite(sigmas).all():
                raise ValueError(sigmas)
            color = next(color_cycle)
            ax.hist(
                sigmas, bins=bins, 
                histtype='step',
                color=color,
                label=r'Testing: {}'.format(key) + ' [$\sigma_{50\%}$ = ' + '{:3.2f}]'.format(np.median(sigmas)),
            )
            ax.axvline(np.median(sigmas), color=color, ls='--')

        ax.plot(np.inf, np.inf, color='0.7', ls='--', label='Median $\sigma$')
        ax.legend(loc='upper left')
        ax.set_ylabel('Frequency')

    axes[-1].set_xlabel('Significance in $\sigma$')

    fig.tight_layout()
    return fig

if False:
    fig = plot_model_confusion(template_keys=['pi0', 'kra5', 'kra50'])
    fig.savefig('{}/model_confusion_nsigma_{:3.2f}.png'.format(plot_dir, nsigma))

fig = plot_model_confusion(template_keys=list(tr_dict.keys()))
fig.savefig('{}/model_confusion_nsigma_{:3.2f}_all.png'.format(plot_dir, nsigma))

fig = plot_model_confusion(
    template_keys=['GalacticCenter', 'GalacticPlane', 'pi0_gamma2', 'pi0_gamma3_10TeV'], 
    testing_keys=['pi0', 'kra5', 'kra50'],
)
fig.savefig('{}/model_confusion_nsigma_{:3.2f}_sys.png'.format(plot_dir, nsigma))



#### Scatter Plot between KRA-gamma 5 PeV and Pi0

In [ ]:

for m1, m2 in [['pi0', 'kra5'], ['pi0', 'kra50'], ['kra5', 'kra50']]:
    
    fig, axes = plt.subplots(2, 1, figsize=(6, 8), sharex=True)
    bins = (np.linspace(1., 6., 15), np.linspace(1., 6., 15))

    # injection of pi0
    axes[0].hist2d(sigma_dict[(m1, m1)], sigma_dict[(m1, m2)], bins=bins, cmin=1)
    axes[0].set_title('Injection template: {}'.format(m1))

    axes[1].hist2d(sigma_dict[(m2, m1)], sigma_dict[(m2, m2)], bins=bins, cmin=1)
    axes[1].set_title('Injection template: {}'.format(m2))

    axes[0].set_ylabel('$n\cdot \sigma$ tested with {}'.format(m2))
    axes[1].set_ylabel('$n\cdot \sigma$ tested with {}'.format(m2))
    axes[-1].set_xlabel('$n\cdot \sigma$ tested with {}'.format(m1))

    for ax in axes:
        ax.plot((bins[0][0], bins[0][-1]), (bins[0][0], bins[0][-1]), ls='--', color='0.8', lw=2)

    fig.tight_layout()
    fig.savefig('{}/model_confusion_correlation_{}_{}_nsigma_{:3.2f}.png'.format(plot_dir, m1, m2, nsigma))
    
    